In [1]:
from pathlib import Path
import os

from transformers import pipeline
from transformers import RobertaTokenizer, RobertaModel, AutoModel, AutoTokenizer
import pandas as pd
import torch

# from data import Dataloader_finetuning
from data.dataloader_finetuning import VaalikoneDataset, VaalikoneDataModule
from models.model import VaalikoneClassifier
from config.finetune_config import config

# Clean vaalit_2019.csv and save it to data/finetune.csv if it doesn't exist
data_path = "data/finetune.csv"
if not os.path.exists(data_path):
    print("Cleaning data")
    df = pd.read_csv("data/vaalit_2019.csv")
    
    # Remove rows with empty strings
    lappi_column_names = [col for col in df.columns if col[:6] == 'Lappi.']
    df = df.dropna(subset=lappi_column_names)
    # df = df[df.apply(lambda x: x.str.len().gt(0).all(), axis=1)]

    # df.to_csv(data_path, index=False)
    df.to_csv(data_path)

# Create training, validation and test sets if they don't exist
train_path = "data/finetune_train.csv"
val_path = "data/finetune_val.csv"
test_path = "data/finetune_test.csv"

if not os.path.exists(train_path) or not os.path.exists(val_path) or not os.path.exists(test_path):
    print("Creating train, val and test sets")
    df = pd.read_csv(data_path)
    # df = df.sample(frac=1).reset_index(drop=True)
    
    # Split the data into train, val and test sets
    train_df = df.iloc[:int(len(df) * 0.8)]
    val_df = df.iloc[int(len(df) * 0.8):int(len(df) * 0.9)]
    test_df = df.iloc[int(len(df) * 0.9):]
    
    train_df.to_csv(train_path, index=False)
    val_df.to_csv(val_path, index=False)
    test_df.to_csv(test_path, index=False)

### Test dataset module
# Load data
ds_train = VaalikoneDataset(
    path=config["train_path"],
    config=config)

# dl_train.__getitem__(1)

# ### Test pytorch-lightning module
# datamodule = VaalikoneDataModule(
#     train_path=config["train_path"],
#     val_path=config["val_path"],
#     test_path=config["test_path"],
#     tokenizer=tokenizer,
#     config=config)
# datamodule.setup()
# dl_train2 = datamodule.train_dataloader()

# print(len(dl_train2))
# print(len(dl_train))

classifier = VaalikoneClassifier(config=config)

/home/nicklas/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'config.config'

In [ ]:
idx = 1
item = ds_train.__getitem__(idx)
input_ids = item["input_ids"]
attention_mask = item["attention_mask"]
labels = item["labels"]
loss, logits = classifier(input_ids, attention_mask)

# Softmax
probs = torch.softmax(logits, dim=1)
print(probs)
print(ds_train.data.iloc[idx])

tensor([[0.1183, 0.0804, 0.5460, 0.2553],
        [0.1697, 0.0753, 0.1503, 0.6047],
        [0.1797, 0.0735, 0.3363, 0.4104],
        [0.3192, 0.2329, 0.1303, 0.3176],
        [0.2078, 0.0707, 0.5011, 0.2203]], grad_fn=<SoftmaxBackward0>)
L_1                                                     5
L_2                                                     5
L_3                                                     1
L_4                                                     4
L_5                                                     1
L_6     Kaivospaikkakunnat pitää saada hyötymään enemm...
L_7     Työtehtävät voivat olla ympäri Lappia, tietoli...
L_8     Lappi on laaja maakunta. Lappia ei voi asettaa...
L_9                                       Ikuisuuskysymys
L_10    Meri-Lapin satamien kautta tavara ulkomaille. ...
Name: 1, dtype: object


In [ ]:
input_ids, labels, input_ids.shape

(tensor([[  102, 37436,  2919, 10463,   751,  1202,  8268,   539,   947, 10002,
           7452,   103],
         [  102, 34484, 48922, 50008,  2046,   439,  2525, 48555,   119, 31177,
          23184,   103],
         [  102, 25100,   145,  5672,  8115,   111, 48555,   193,   326,  8242,
           3140,   103],
         [  102,   320, 37052, 25849,  2809,  2565,  7572,   103,     0,     0,
              0,     0],
         [  102, 13310,   166,  5333,  2898,  1477,  1315,  8810, 10287,   111,
           2597,   103]]),
 [5, 5, 1, 4, 1],
 torch.Size([5, 12]))